In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import pickle
import numpy as np

In [4]:
model = load_model('model.h5')

2024-10-23 18:03:03.452444: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-10-23 18:03:03.452482: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-10-23 18:03:03.452489: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-10-23 18:03:03.452904: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-23 18:03:03.453374: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
with open('onehot_encoder_geo.pkl','rb') as file:
    geo_encode = pickle.load(file)
with open('label_encoder_gender.pkl','rb') as file:
    gender_encode = pickle.load(file)
with open('scaler.pkl','rb') as file:
    scale = pickle.load(file)

/Users/shubhamgadhwal/Downloads/ann/venv/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.5.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/shubhamgadhwal/Downloads/ann/venv/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/shubhamgadhwal/Downloads/ann/venv/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardS

In [6]:
input_data = {
    'CreditScore' : 600,
    'Geography' : 'France',
    'Gender' : 'Male',
    'Age' : 40,
    'Tenure' : 3,
    'Balance' : 60000,
    'NumOfProducts' : 2,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' : 50000
 }

In [10]:
geo_encoded = geo_encode.transform([[input_data['Geography']]]).toarray()
geo_encodeddf = pd.DataFrame(geo_encoded,columns=geo_encode.get_feature_names_out(['Geography']))

/Users/shubhamgadhwal/Downloads/ann/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [11]:
geo_encodeddf

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [13]:
input_df = pd.DataFrame([input_data])

In [14]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [15]:
input_df['Gender']=gender_encode.transform(input_df['Gender'])

In [16]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [17]:
input_df = pd.concat([input_df.drop('Geography',axis=1),geo_encodeddf],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [23]:
input_scaled=scale.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [21]:
prediction=model.predict(input_scaled)

1/1 [==============================] - 0s 12ms/step


In [22]:
prediction

array([[6.645533e-09]], dtype=float32)

In [24]:
prediction_proba = prediction[0][0]
if prediction_proba > 0.5:
    print("Customer lodu")
else:
    print("customer lodu bhangi")

customer lodu bhangi
